In [1]:
number_of_shots = 7

In [2]:
experiment_name = "2024.05.07-03_Mixtral_7B_instruct_API_7-few-shots_RANDOM"

In [3]:
model_name = "mistralai/Mixtral-8x7B-Instruct-v0.1"

In [4]:
!pip install -q openai #--progress-bar off

In [5]:
import pandas as pd

import datetime
from langchain import PromptTemplate
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import PromptTemplate

import os
import json
import ast
import re

import pickle

import glob

from typing import Union
from IPython.display import display, HTML

from sklearn.metrics import classification_report

In [6]:
import os
from IPython.display import clear_output

os.environ["OPENAI_API_BASE"] = 'https://api.endpoints.anyscale.com/v1'
os.environ["OPENAI_API_KEY"] = input("INSERT YOUR ANYSCALE API KEY:")

clear_output()

In [7]:
# output_datapath = "data/output/2023.11.16-1_Mistral_7B_instruct_API/"
# final_report_name = 'report_sdg_classification_Mistral_7B.xlsx'
output_datapath = f"data/output/{experiment_name}/"
log_filename = f"logs/{experiment_name}.log"

final_report_name = 'report_sdg_classification_Mixtral_7B.xlsx'

In [8]:
import os
import requests

s = requests.Session()

api_base = os.getenv("OPENAI_API_BASE")
token = os.getenv("OPENAI_API_KEY")
url = f"{api_base}/chat/completions"

In [9]:
data_sel_4_few_shot_examples = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_4760_texts_TRAIN_2023.12.11.xlsx").drop(columns=['Unnamed: 0'])
data_sel_4_few_shot_examples = data_sel_4_few_shot_examples[
    (data_sel_4_few_shot_examples.text_id==5614) |
    (data_sel_4_few_shot_examples.text_id==2089) |
    (data_sel_4_few_shot_examples.text_id==5665) |
    (data_sel_4_few_shot_examples.text_id==370)  |
    (data_sel_4_few_shot_examples.text_id==1422)  |
    (data_sel_4_few_shot_examples.text_id==4329) |
    (data_sel_4_few_shot_examples.text_id==4929) 

]

data_sel_4_few_shot_examples

,Unnamed: 0.1,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length
738,738,370,10.1787/9789264272392-3-en,"For example, the poorest 20% of farm household...",2,0.0,9.0,1.000000,96
1092,1092,1422,10.1787/9789264283299-en,"Nevertheless, the average number of physician ...",3,1.0,8.0,0.777778,76
2319,2319,5665,10.18356/3fe10a08-en,"As a result, both the trade and current accoun...",8,0.0,5.0,1.000000,53
3569,3569,2089,10.18356/ee52a573-en,"This refers to the manufacture, use and dispos...",12,3.0,29.0,0.812500,47
3883,3883,4929,10.18356/5e8977eb-en,"Hence, climate-economy models without explicit...",13,0.0,3.0,1.000000,92
4451,4451,4329,10.18356/30e5b8d3-en,With the introduction of FSC standards in the ...,15,0.0,9.0,1.000000,114
4608,4608,5614,10.2139/SSRN.1406042,The modern definitions of the inchoate offence...,16,0.0,6.0,1.000000,136


In [10]:
few_shots = ""

for _, row in data_sel_4_few_shot_examples[['text', 'sdg']].iterrows():
    input_text = "Input text: \""+row['text']+"\"\n"
    output_text = "Output: \"---SDG "+str(row['sdg'])+'---\"\n\n'
    
    few_shots += input_text
    few_shots += output_text
    
print(few_shots)

Input text: "For example, the poorest 20% of farm households in the Philippines tend to be small-scale producers and landless workers whose rice production does not satisfy their own needs and who therefore purchase 77% of the rice that they consume on average (Chapter 4). Regional levels of stunting among children under five years of age, for example, remain worryingly high (Figure 3). While economic growth has enabled significant declines in populations suffering from chronic poverty, for some households, it may have only altered the nature of food insecurity from permanent to transitory, rather than eliminate it entirely."
Output: "---SDG 2---"

Input text: "Nevertheless, the average number of physician contacts per person is not greater than the EU average. For example, home visits to patients by GPs are a regular practice and patients do not usually have to wait long to get access to GPs, although waiting times for more specialised services (e.g. mental health specialists, ophthal

In [11]:
data = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_1020_texts_TEST_2023.12.11.xlsx")
data = data.drop(columns=['Unnamed: 0'])
data.head(3)

,Unnamed: 0.1,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length
0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,38
1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,15
2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,125


In [12]:
# template_part1 = """[INST]
# <<SYS>>
# In the following lines, a text enclosed in triple quotes will be given to you.

# The task is to classify the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015.

# The constraint to adhere to is that for the classification, you must select only one out of all the possible SDGs.

# If you believe that the text to be processed cannot be classified under any of the possible SDGs, then classify it as "SDG 0".

# The text to be processed is:
# <</SYS>>

# '''
# """

# template_part2 = """
# '''

# <<SYS>>
# Answer with the label only using the format:---LABEL---.
# <</SYS>>
# [/INST]
# """

system_prompt = f"""In the following lines, a text enclosed in triple quotes will be given to you.

The task is to classify the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015.

The constraint to adhere to is that for the classification, you must select only one out of all the possible SDGs.

If you believe that the text to be processed cannot be classified under any of the possible SDGs, then classify it as "---SDG 0---".

The following are {number_of_shots} examples of expected classification given an input text:

"""+few_shots+"""

Answer with the label only using the format:---LABEL---
"""


In [13]:
data_sel = data #.iloc[:5]

In [14]:
import time

# cnt = 0
results = []
for index, row in data_sel.iterrows():
    
    cnt = str(index)
    
    file = open(log_filename, 'a+', encoding='utf-8')
    file.write("\n\n##################################################\n")
    
    text_id = row['text_id']
    text = row["text"]
    sdg_true = row["sdg"]
    
    #query = template_part1 + text + template_part2
    query = text
    
    try:
        body = {
          "model": model_name,
          "messages": [
              {"role": "system", "content": system_prompt}, 
              {"role": "user", "content": query}],
          "temperature": 0.0
        }

        result = s.post(url, headers={"Authorization": f"Bearer {token}"}, json=body).json()
        print(cnt, text_id, sdg_true, result['choices'][0]['message']['content'])
        file.write(str(cnt)+" "+str(text_id)+" "+str(sdg_true)+str(result)+"\n")
    except Exception as e:
        result = str(e)
        print(cnt, text_id, sdg_true, "ERROR: "+result)
        file.write(str(cnt)+" "+str(text_id)+" "+str(sdg_true)+" "+"ERROR: "+result+"\n")        
    
#     print(cnt, text_id, sdg_true, result['choices'][0]['message']['content'])
    
    result_extended = (cnt, text_id, sdg_true, result)
    #print(result_extended)
    
    with open(output_datapath+f'sdg_classification_{model_name.replace("/","_")}_{cnt}.pkl', 'wb') as f:
        pickle.dump(result_extended, f)
    
    results.append(result_extended)
    #cnt+=1
    
    file.close()
    
    time.sleep(0.5)
    
    

0 6668 0  ---SDG 0--- (This text does not seem to fit any specific SDG and discusses a subway derailment, which is not directly related to any of the SDGs.)
1 6099 0  ---SDG 0---

The text is about a model's photo shoot for a magazine and does not relate to any specific SDG.
2 6471 0  ---SDG 16---

The text discusses the legal system and the fight against terrorism, which aligns with SDG 16: Peace, Justice and Strong Institutions. Specifically, it talks about the suspension of a life sentence in a terror-related case and the impact of the fight against terrorism on inchoate offenses.
3 1786 0  ---SDG 4--- (Goal 4: Quality Education) This passage discusses the naming of minor planets after four Indian scientists, including a Malayali, which highlights the importance of education and recognition of individuals' contributions in the field of science.
4 2143 0  ---SDG 7---
5 3377 0  ---SDG 5--- (Gender Equality) based on the theme of women's experiences and empowerment in the text.
6 5589 

53 4104 0  ---SDG 3---
54 1497 0  ---SDG 8---
55 2055 0  ---SDG 3--- (goal for good health and well-being) as the text discusses medical topics such as cochlea cells and single-cell sequencing.
56 2463 0  ---SDG 0---

The text does not appear to be related to any specific Sustainable Development Goal. It is about football team promotions and does not contain any information that would classify it under SDG 1 (No Poverty), SDG 2 (Zero Hunger), SDG 3 (Good Health and Well-being), SDG 4 (Quality Education), SDG 5 (Gender Equality), SDG 6 (Clean Water and Sanitation), SDG 7 (Affordable and Clean Energy), SDG 9 (Industry, Innovation and Infrastructure), SDG 10 (Reduced Inequalities), SDG 11 (Sustainable Cities and Communities), SDG 13 (Climate Action), SDG 14 (Life Below Water), SDG 16 (Peace, Justice and Strong Institutions), or SDG 17 (Partnerships for the Goals).
57 193 0  ---SDG 0--- (The text does not seem to be related to any specific SDG and does not contain information that would al

137 2747 2  ---SDG 2---
138 3976 2  ---SDG 2--- (This text is about agriculture and food production, which aligns with SDG 2's focus on ending hunger, achieving food security, improving nutrition, and promoting sustainable agriculture.)
139 2988 2  ---SDG 2---
140 2775 2  ---SDG 2---
141 5423 2  ---SDG 2---
142 5748 2  ---SDG 2---
143 1642 2  ---SDG 2---
(This text is about food supply and redistribution, which relates to the goal of Zero Hunger (SDG 2).)
144 2727 2  ---SDG 2---
145 869 2  ---SDG 2---
146 3205 2  ---SDG 2---
147 6506 2  ---SDG 2---
148 1622 2  ---SDG 2---
149 2827 2  ---SDG 8---
150 223 2  ---SDG 2---
151 1319 2  ---SDG 2---
152 5700 2  ---SDG 1---
(Referring to SDG 1: No Poverty, as the text discusses social transfers, community-based insurance, and school feeding programs to improve the lives of food-insecure people, although not explicitly linked to food security.)
153 977 2  ---SDG 8---
154 3039 2  ---SDG 2---
155 6419 2  ---SDG 2---
156 5694 2  ---SDG 2---
(The te

240 627 4  ---SDG 4---
241 1661 4  ---SDG 4---
242 3883 4  ---SDG 4---
243 1181 4  ---SDG 3--- (Goal 3 is about ensuring healthy lives and promoting well-being for all at all ages)
244 2920 4  ---SDG 4---
245 6256 4  ---SDG 4---
246 3956 4  ---SDG 4--- (Quality Education)
247 6094 4  ---SDG 4---
248 5231 4  ---SDG 4--- (Quality Education)
249 2573 4  ---SDG 4--- (Quality Education)
250 6454 4  ---SDG 4---
251 5999 4  ---SDG 4--- (Goal 4 is Quality Education)
252 5448 4  ---SDG 4--- (Quality Education)
253 5380 4  ---SDG 4--- (Quality Education)
254 2608 4  ---SDG 4--- (Quality Education)
255 823 4  ---SDG 4--- (Quality Education)
256 1972 4  ---SDG 4---
257 3492 4  ---SDG 4--- (Quality Education)
258 218 4  ---SDG 4--- (Quality Education)
259 3447 4  ---SDG 4---
260 3865 4  ---SDG 4--- (Goal 4 is about ensuring inclusive and equitable quality education and promoting lifelong learning opportunities for all.) The text highlights the negative impact of the Syrian conflict on the education

375 3955 6  ---SDG 6--- (Clean Water and Sanitation) as the text discusses the importance and efficiency of water trading in agriculture, which relates to the management and distribution of water resources.
376 4253 6  ---SDG 6--- (Clean Water and Sanitation)
377 5477 6  ---SDG 6--- (Clean Water and Sanitation)
378 3247 6  ---SDG 6--- (Clean Water and Sanitation)
379 598 6  ---SDG 17---

The text discusses the misconceptions around the concept of virtual water and its relationship with the economic theory of comparative advantage. It highlights how the application of virtual water aligns more with the concept of absolute advantage, which may not result in the maximization of net benefits in international trade. This topic relates to international trade policies and global partnerships for sustainable development (SDG 17).
380 4015 6  ---SDG 6--- (Clean Water and Sanitation)
381 961 6  ---SDG 6--- (Clean Water and Sanitation) is the most appropriate SDG for this text as it discusses the

...

799 1620 13  ---SDG 13---
800 3379 13  ---SDG 13---
801 4843 13  ---SDG 13---
802 6675 13  ---SDG 17---
(This text discusses the mobilization of external resources for development, which aligns with SDG 17's focus on strengthening global partnerships and financing for sustainable development.)
803 6696 13  ---SDG 13---

---

Explanation: The text discusses the importance of accounting for climate risk in financial evaluations, which aligns with SDG 13 (Climate Action) as it emphasizes the need to take urgent action to combat climate change and its impacts.
804 5030 13  ---SDG 13--- (as the text discusses climate change mitigation and transparency framework for sharing information, which aligns with SDG 13: Climate Action)
805 5047 13  ---SDG 13---
(This text discusses the importance of concrete mechanisms for action in addressing climate change, which aligns with SDG 13: Climate Action.)
806 220 13  ---SDG 13--- (Climate Action)
807 1613 13  ---SDG 12---
(This text is about certificati

887 5734 14  ---SDG 14---
888 1089 14  ---SDG 14---
889 5290 14  ---SDG 14---
890 5720 14  ---SDG 14---
891 204 14  ---SDG 14---
892 6541 14  ---SDG 14---
893 2042 14  ---SDG 16---

The text discusses the establishment of a community quota clause in the Fisheries Management Act to promote equality and community rights. This is related to SDG 16, which aims to promote peaceful and inclusive societies for sustainable development, provide access to justice for all and build effective, accountable and inclusive institutions at all levels.
894 3687 14  ---SDG 2---
895 5519 14  ---SDG 14---
896 3637 14  ---SDG 14--- (Sustainable Development Goal 14 is about Life Below Water, which includes the conservation and sustainable use of the oceans, seas and marine resources for sustainable development.)
897 4240 14  ---SDG 14---
898 640 14  ---SDG 14---
899 3336 14  ---SDG 14--- (Life Below Water)
900 893 15  ---SDG 13---
901 6606 15  ---SDG 13---

The text discusses the importance of considering ec

951 2066 15  ---SDG 15---
952 389 15  ---SDG 15---

This text is about biodiversity in Scotland and the importance of preserving it, which aligns with the targets of Sustainable Development Goal 15 (Life on Land), specifically target 15.5 which aims to "take urgent and significant action to reduce the degradation of natural habitats, halt the loss of biodiversity and, by 2020, protect and prevent the extinction of threatened species." The text mentions the loss of biodiversity in Scotland and the need to protect it, which is why it is classified under SDG 15.
953 943 15  ---SDG 15--- (This text is about protecting, restoring and promoting the sustainable use of terrestrial ecosystems, specifically about the loss of vertebrate and plant species populations and the presence of invasive species, which aligns with SDG 15: Life on Land)
954 6522 15  ---SDG 15---
955 3665 15  ---SDG 15---

This text is about the conservation and monitoring of biodiversity in Turkmenistan's State reserves, wh

987 2092 16  ---SDG 5---
988 2472 16  ---SDG 12--- (This text is about responsible consumption and production, which is related to SDG 12. The text discusses the role of investors in funding projects that may cause ecological degradation or social injustice, and the use of divestment and responsible investment strategies by NGOs to bring about change.)
989 3167 16  ---SDG 16---

The text discusses the development and validation of the Support for Diplomacy Scale (SDS), which measures attitudes towards peacemaking and militarism. The SDS is associated with various political and social attitudes, and it has been shown to mediate the relation between just peacemaking and various measures of militarism and resistance to moral disengagement from war. The SDS also predicts voting behavior and mediates the relation between political orientation and foreign policy attitudes. These topics are related to SDG 16, which aims to promote peaceful and inclusive societies, provide access to justice fo

1013 6330 16  ---SDG 16---
1014 2723 16  ---SDG 16---

The article discusses the neo-conservative critique of global liberal governance, which is a political and ethical issue related to the functioning of democratic institutions, human rights, and international law. This topic falls under SDG 16, which promotes peaceful and inclusive societies for sustainable development, provides access to justice for all, and builds effective, accountable, and inclusive institutions at all levels.
1015 4695 16  ---SDG 16---
1016 6628 16  ---SDG 17---

Explanation:
The text discusses the creation of global institutions to address public good issues, which includes international cooperation and the establishment of international norms and treaties. This aligns with SDG 17, which is about strengthening global partnerships to achieve sustainable development goals.
1017 4563 16  ---SDG 16---
1018 6710 16  ---SDG 16---

The text discusses the due process protections for plaintiff class members in class ac

In [15]:
files = glob.glob(output_datapath + "*.pkl")
# files

In [16]:
# df_list = (pd.DataFrame(pd.read_pickle(file), columns=['cnt', 'id', 'true_label', 'predicted_label']) for file in files)
# df_list = (pd.DataFrame(pd.read_pickle(file)) for file in files)

tuple_list = []
for file in files:
    res = pd.read_pickle(file)
    if len(res[3]['choices'])>1:
        raise Exception('ERROR: Multilabel detected!')
    sdg_pred = res[3]['choices'][0]['message']['content']
    res4df = (res[0], res[1], res[2], sdg_pred)
    print(res4df)
    tuple_list.append(res4df)

('1015', 4695, 16, ' ---SDG 16---')
('582', 714, 9, ' ---SDG 9--- (This text is about strengthening infrastructure and promoting innovation in the field of science, technology, and innovation (STI), which aligns with SDG 9.)')
('2', 6471, 0, ' ---SDG 16---\n\nThe text discusses the legal system and the fight against terrorism, which aligns with SDG 16: Peace, Justice and Strong Institutions. Specifically, it talks about the suspension of a life sentence in a terror-related case and the impact of the fight against terrorism on inchoate offenses.')
('782', 2180, 13, ' ---SDG 13---')
('601', 4947, 10, ' ---SDG 10---')
('144', 2727, 2, ' ---SDG 2---')
('54', 1497, 0, ' ---SDG 8---')
('125', 5756, 2, ' ---SDG 11--- (This text pertains to making cities and human settlements inclusive, safe, resilient, and sustainable, which is aligned with SDG 11.)')
('611', 3928, 10, ' ---SDG 10--- (Reduced Inequalities) as the text discusses issues of income inequality and redistribution policies.')
('354'

In [17]:
results_processed = pd.DataFrame(tuple_list, columns=['cnt', 'id', 'true_label', 'predicted_label'])

In [18]:
results_processed

,cnt,id,true_label,predicted_label
0,1015,4695,16,---SDG 16---
1,582,714,9,---SDG 9--- (This text is about strengthening...
2,2,6471,0,---SDG 16---\n\nThe text discusses the legal ...
3,782,2180,13,---SDG 13---
4,601,4947,10,---SDG 10---
...,...,...,...,...
1015,152,5700,2,"---SDG 1---\n(Referring to SDG 1: No Poverty,..."
1016,276,1547,4,---SDG 4--- (Quality Education)
1017,554,5840,9,---SDG 8---\n(This text is discussing economi...
1018,927,6479,15,---SDG 15---\n(This text is about the certifi...


In [19]:
results_processed_merged = data.merge(results_processed, right_on='id', left_on='text_id')
results_processed_merged.head(3)

,Unnamed: 0.1,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length,cnt,id,true_label,predicted_label
0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,38,0,6668,0,---SDG 0--- (This text does not seem to fit a...
1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,15,1,6099,0,---SDG 0---\n\nThe text is about a model's ph...
2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,125,2,6471,0,---SDG 16---\n\nThe text discusses the legal ...


In [20]:
results_processed_merged['predicted_label_clean'] = results_processed_merged['predicted_label'].str.extract(r'SDG (\d+)')
results_processed_merged.head(3)

,Unnamed: 0.1,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length,cnt,id,true_label,predicted_label,predicted_label_clean
0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,38,0,6668,0,---SDG 0--- (This text does not seem to fit a...,0
1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,15,1,6099,0,---SDG 0---\n\nThe text is about a model's ph...,0
2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,125,2,6471,0,---SDG 16---\n\nThe text discusses the legal ...,16


In [21]:
results_processed_merged.groupby("predicted_label_clean").count()

,Unnamed: 0.1,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length,cnt,id,true_label,predicted_label
predicted_label_clean,,,,,,,,,,,,,
0,16,16,1,16,16,1,1,1,16,16,16,16,16
1,71,71,71,71,71,71,71,71,71,71,71,71,71
10,32,32,32,32,32,32,32,32,32,32,32,32,32
11,86,86,79,86,86,79,79,79,86,86,86,86,86
12,53,53,53,53,53,53,53,53,53,53,53,53,53
13,59,59,58,59,59,58,58,58,59,59,59,59,59
14,53,53,53,53,53,53,53,53,53,53,53,53,53
15,59,59,57,59,59,57,57,57,59,59,59,59,59
16,50,50,45,50,50,45,45,45,50,50,50,50,50


In [22]:
results_processed_merged.to_excel(output_datapath+final_report_name)

In [23]:
import numpy as np
from sklearn.metrics import classification_report

y_true = results_processed_merged.sdg 
y_pred = results_processed_merged.predicted_label_clean
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

1020
1020
              precision    recall  f1-score   support

           0       0.94      0.25      0.39        60
           1       0.70      0.83      0.76        60
           2       0.88      0.82      0.84        60
           3       0.75      0.95      0.84        60
           4       0.84      0.93      0.88        60
           5       0.85      0.92      0.88        60
           6       0.98      0.82      0.89        60
           7       0.84      0.90      0.87        60
           8       0.51      0.73      0.60        60
           9       0.74      0.72      0.73        60
          10       0.91      0.48      0.63        60
          11       0.67      0.97      0.79        60
          12       0.87      0.77      0.81        60
          13       0.80      0.78      0.79        60
          14       0.96      0.85      0.90        60
          15       0.90      0.88      0.89        60
          16       0.86      0.72      0.78        60
          17     

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [24]:
# AGGIUNTO IL 2024.05.07:

In [25]:
# CORREZIONI NEL CASO IN CUI IL MODELLO RESTITUISCA DUPLICATI O OUTPUT NON UTILI PER DEFINIRE LA CLASSE O CLASSI NON PREVISTE 

import random

# NEL CASO IN CUI IL MODELLO RESTITUISCA DUPLICATI SELEZIONO SEMPRE IL PRIMO
results_processed_merged = results_processed_merged.drop_duplicates(subset='text_id', keep='first')

# # NEL CASO IN CUI IL MODELLO RESTITUISCA OUTPUT NON UTILI PER DEFINIRE LA CLASSE SI APPLICA LA CLASSE 0
# # results_processed_merged.loc[results_processed_merged['predicted_label_clean'].isna(), 'predicted_label_clean'] = 0
# results_processed_merged.loc[results_processed_merged['predicted_label_clean'].isna(), 'predicted_label_clean'] = results_processed_merged.loc[results_processed_merged['predicted_label_clean'].isna(), 'sdg'] - 1

# # NEL CASO IN CUI IL MODELLO RESTITUISCA CLASSI NON PREVISTE SI APPLICA LA CLASSE 0
# results_processed_merged['predicted_label_clean'] = results_processed_merged.predicted_label_clean.apply(lambda x: 0 if int(x)>16 else x)

# Lista di valori sdg_list
sdg_list = [str(x) for x in range(17)]
# sdg_list = [x for x in range(17)]
# print(sdg_list)

# Se il valore di 'predicted_label_clean' e' NaN o un valore numerico non presente nella lista sdg_list 
# allora assegna a  'predicted_label_clean' un valore qualunque estratto a caso da sdg_list ma diverso dal valore di 'sdg' per la riga in esame
# Modifica della colonna 'predicted_label_clean' secondo le regole specificate
for index, row in results_processed_merged.iterrows():
    if pd.isna(row['predicted_label_clean']) or row['predicted_label_clean'] not in sdg_list:
#         new_value = random.choice([x for x in sdg_list if x != row['sdg']])
        results_processed_merged.at[index, 'predicted_label_clean'] = 0  # new_value

In [26]:
results_processed_merged.head(3)

,Unnamed: 0.1,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length,cnt,id,true_label,predicted_label,predicted_label_clean
0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,38,0,6668,0,---SDG 0--- (This text does not seem to fit a...,0
1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,15,1,6099,0,---SDG 0---\n\nThe text is about a model's ph...,0
2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,125,2,6471,0,---SDG 16---\n\nThe text discusses the legal ...,16


In [27]:
import numpy as np
from sklearn.metrics import classification_report

y_true = results_processed_merged.sdg 
y_pred = results_processed_merged.predicted_label_clean
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

1020
1020
              precision    recall  f1-score   support

           0       0.49      0.28      0.36        60
           1       0.70      0.83      0.76        60
           2       0.88      0.82      0.84        60
           3       0.75      0.95      0.84        60
           4       0.84      0.93      0.88        60
           5       0.85      0.92      0.88        60
           6       0.98      0.82      0.89        60
           7       0.84      0.90      0.87        60
           8       0.51      0.73      0.60        60
           9       0.74      0.72      0.73        60
          10       0.91      0.48      0.63        60
          11       0.67      0.97      0.79        60
          12       0.87      0.77      0.81        60
          13       0.80      0.78      0.79        60
          14       0.96      0.85      0.90        60
          15       0.90      0.88      0.89        60
          16       0.86      0.72      0.78        60

    accuracy    

In [28]:
classification_report_filename = output_datapath+f'/{experiment_name}_classification_report.xlsx'
classification_report_filename

report=classification_report(y_true, y_pred, output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df[['precision', 'recall', 'f1-score']] *= 100
report_df.to_excel(classification_report_filename)

In [29]:
report_df

,precision,recall,f1-score,support
0,48.571429,28.333333,35.789474,60.000000
1,70.422535,83.333333,76.335878,60.000000
2,87.500000,81.666667,84.482759,60.000000
3,75.000000,95.000000,83.823529,60.000000
4,83.582090,93.333333,88.188976,60.000000
5,84.615385,91.666667,88.000000,60.000000
6,98.000000,81.666667,89.090909,60.000000
7,84.375000,90.000000,87.096774,60.000000
8,51.162791,73.333333,60.273973,60.000000
9,74.137931,71.666667,72.881356,60.000000


In [30]:
accuracy = report_df.loc['accuracy']['precision']
precision = report_df.loc['macro avg']['precision']
recall = report_df.loc['macro avg']['recall']
f1_score = report_df.loc['macro avg']['f1-score']

In [31]:
print(precision, recall, accuracy, f1_score)

79.64378902916765 78.52941176470588 78.52941176470588 78.02291893139892


In [32]:
print(round(precision, 1), round(recall, 1), round(accuracy, 1), round(f1_score, 1))

79.6 78.5 78.5 78.0
